In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nnfl-lab-3/lab3_test.csv
/kaggle/input/nnfl-lab-3/sample_submission.csv
/kaggle/input/nnfl-lab-3/lab3_train.csv


In [2]:
df = pd.read_csv('../input/nnfl-lab-3/lab3_train.csv')

In [3]:
EMBEDDING_DIM=200
import string 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

txt = list()
lines = df['text'].values.tolist()

for line in lines:
    tokens = word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    txt.append(words)
txt

[['due',
  'to',
  'the',
  'large',
  'numbers',
  'of',
  'transcription',
  'factors',
  'tfs',
  'and',
  'cell',
  'types',
  'querying',
  'binding',
  'profiles',
  'of',
  'all',
  'valid',
  'tfcell',
  'type',
  'pairs',
  'is',
  'not',
  'experimentally',
  'feasible',
  'to',
  'address',
  'this',
  'issue',
  'we',
  'developed',
  'a',
  'convolutionalrecurrent',
  'neural',
  'network',
  'model',
  'called',
  'factornet',
  'to',
  'computationally',
  'impute',
  'the',
  'missing',
  'binding',
  'data',
  'factornet',
  'trains',
  'on',
  'binding',
  'data',
  'from',
  'reference',
  'cell',
  'types',
  'to',
  'make',
  'predictions',
  'on',
  'testing',
  'cell',
  'types',
  'by',
  'leveraging',
  'a',
  'variety',
  'of',
  'features',
  'including',
  'genomic',
  'sequences',
  'genome',
  'annotations',
  'gene',
  'expression',
  'and',
  'signal',
  'data',
  'such',
  'as',
  'dnase',
  'i',
  'cleavage',
  'factornet',
  'implements',
  'several',

In [4]:
import gensim
model = gensim.models.Word2Vec(sentences = txt, size=EMBEDDING_DIM, window = 5, workers = 4,min_count = 1)
words = list(model.wv.vocab)
len(words)

79015

In [5]:
filename = 'embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary = False)

import os
embeddings_index = {}
f = open(os.path.join('', 'embedding_word2vec.txt'), encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [6]:
max_length = max([len(s.split()) for s in (df['text'])])

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(txt)
sequences = tokenizer_obj.texts_to_sequences(txt)

word_index = tokenizer_obj.word_index

txt_pad = pad_sequences(sequences, maxlen=max_length)
sentiment=df['class'].values
sentiment = pd.get_dummies(sentiment)
sentiment

,1,2,3,4,5
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
29995,0,1,0,0,0
29996,0,0,0,0,1
29997,0,0,0,1,0
29998,0,0,0,1,0


In [8]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.initializers import Constant

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix),input_length = max_length,trainable = False))
model.add(SpatialDropout1D(0.2))   
model.add(LSTM(60, dropout=0.2, recurrent_dropout = 0.2,return_sequences = True))
model.add(SpatialDropout1D(0.4))   
model.add(LSTM(120, dropout=0.2, recurrent_dropout = 0.2,return_sequences = False))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1103, 200)         15803200  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 1103, 200)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 1103, 60)          62640     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1103, 60)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 120)               86880     
_________________________________________________________________
dense (Dense)                (None, 5)                 605       
Total params: 15,953,325
Trainable params: 150,125
Non-trainable params: 15,803,200
______________________________________

In [10]:
sentiment = sentiment.to_numpy()

In [11]:
VALIDATION_SPLIT = 0.2
indices = np.arange(txt_pad.shape[0])
np.random.shuffle(indices)
txt_pad = txt_pad[indices]
sentiment = sentiment[indices]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(txt_pad, sentiment, test_size=0.2, random_state=42)

In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
batch_size = 512
history = model.fit(X_train, y_train, epochs = 30, batch_size=batch_size, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss',patience = 5, min_delta = 0.001)])

Epoch 1/30
38/38 [==============================] - 308s 8s/step - loss: 0.3436 - accuracy: 0.8896 - val_loss: 0.2716 - val_accuracy: 0.9129
Epoch 2/30
38/38 [==============================] - 306s 8s/step - loss: 0.4068 - accuracy: 0.8647 - val_loss: 0.3316 - val_accuracy: 0.8965
Epoch 3/30
38/38 [==============================] - 308s 8s/step - loss: 0.3525 - accuracy: 0.8901 - val_loss: 0.2674 - val_accuracy: 0.9133
Epoch 4/30
38/38 [==============================] - 308s 8s/step - loss: 0.3280 - accuracy: 0.8904 - val_loss: 0.2652 - val_accuracy: 0.9169
Epoch 5/30
38/38 [==============================] - 310s 8s/step - loss: 0.2850 - accuracy: 0.9089 - val_loss: 0.2316 - val_accuracy: 0.9290
Epoch 6/30
38/38 [==============================] - 307s 8s/step - loss: 0.2795 - accuracy: 0.9121 - val_loss: 0.2322 - val_accuracy: 0.9312
Epoch 7/30
38/38 [==============================] - 310s 8s/step - loss: 0.2605 - accuracy: 0.9211 - val_loss: 0.2277 - val_accuracy: 0.9323
Epoch 8/30
38

In [23]:
model.save_weights('2018A7PS0127G_Lab3.h5')

In [16]:
df_test = pd.read_csv('../input/nnfl-lab-3/lab3_test.csv')

In [17]:
test_sequences = tokenizer_obj.texts_to_sequences(df_test['text'].values)
test_txt_pad = pad_sequences(test_sequences, maxlen=max_length)

In [18]:
predict = model.predict(x=test_txt_pad, verbose = 1)
predict

382/382 [==============================] - 171s 447ms/step


array([[2.1667795e-03, 4.7696455e-04, 9.9229795e-01, 2.1953722e-03,
        2.8630444e-03],
       [9.9600345e-01, 1.8019130e-03, 9.5749222e-04, 8.0223411e-04,
        4.3492374e-04],
       [2.9078012e-03, 3.0580410e-04, 3.4223378e-03, 9.6677190e-01,
        2.6592210e-02],
       ...,
       [8.0572236e-01, 1.2054087e-03, 3.5402605e-03, 2.2723531e-02,
        1.6680844e-01],
       [9.9777126e-01, 8.6587679e-04, 6.2492717e-04, 5.8907556e-04,
        1.4876078e-04],
       [6.1811809e-04, 9.9879164e-01, 1.2455283e-04, 4.3231260e-04,
        3.3416294e-05]], dtype=float32)

In [19]:
ans = []
for arr in predict:
    ind = arr.argmax()
    ind= ind+1
    ans.append(ind)

In [20]:
df_test = df_test['index']
df_test = pd.DataFrame(df_test)
df_test['class'] = ans

In [21]:
df_test.to_csv('2018A7PS0127G.csv',index=False)